# Pytorch로 딥러닝 제대로 배우기-기초
## Part7: Backpropagation & Activations
### 필수 라이브러리 Import

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam, RMSprop, Adagrad

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### 1. Baseline 모델 준비

#### (1) 데이터 준비

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# 공개된 데이터 셋으로부터 학습 데이터 다운로드 
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개된 데이터 셋으로부터 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#### (2) 모델

In [5]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

#### (3) Optimizer & Losses

In [7]:
# Optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# losses
losses = nn.CrossEntropyLoss()

#### (4) 학습 & 테스트 wrapper

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    print("Training mode: {}".format(model.training))
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    print("Training mode: {}".format(model.training))
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### (5) 학습

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.310004  [    0/60000]
loss: 0.953202  [ 6400/60000]
loss: 0.559980  [12800/60000]
loss: 0.695966  [19200/60000]
loss: 0.534250  [25600/60000]
loss: 0.521526  [32000/60000]
loss: 0.490337  [38400/60000]
loss: 0.625797  [44800/60000]
loss: 0.556165  [51200/60000]
loss: 0.527528  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.480913 

Epoch 2
-------------------------------
Training mode: True
loss: 0.358659  [    0/60000]
loss: 0.443967  [ 6400/60000]
loss: 0.327214  [12800/60000]
loss: 0.486116  [19200/60000]
loss: 0.427651  [25600/60000]
loss: 0.415763  [32000/60000]
loss: 0.399579  [38400/60000]
loss: 0.536456  [44800/60000]
loss: 0.478141  [51200/60000]
loss: 0.500677  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.445515 

Epoch 3
-------------------------------
Training mode: True
loss: 0.311268  [    0/60000]
loss: 0.375889  [ 6400/60000]
loss: 0.292033

### Activation Function 변경

#### (1) Sigmoid

In [11]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [13]:
# Optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# losses
losses = nn.CrossEntropyLoss()

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.349305  [    0/60000]
loss: 2.361865  [ 6400/60000]
loss: 2.286061  [12800/60000]
loss: 1.952526  [19200/60000]
loss: 1.417367  [25600/60000]
loss: 1.244827  [32000/60000]
loss: 1.147588  [38400/60000]
loss: 1.044546  [44800/60000]
loss: 1.052436  [51200/60000]
loss: 0.945761  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 64.3%, Avg loss: 0.908880 

Epoch 2
-------------------------------
Training mode: True
loss: 0.972734  [    0/60000]
loss: 0.935077  [ 6400/60000]
loss: 0.715026  [12800/60000]
loss: 0.865668  [19200/60000]
loss: 0.772313  [25600/60000]
loss: 0.692138  [32000/60000]
loss: 0.790359  [38400/60000]
loss: 0.741537  [44800/60000]
loss: 0.711743  [51200/60000]
loss: 0.713575  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.675297 

Epoch 3
-------------------------------
Training mode: True
loss: 0.606725  [    0/60000]
loss: 0.709137  [ 6400/60000]
loss: 0.451478

#### (2) tanh

In [15]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [16]:
# Optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# losses
losses = nn.CrossEntropyLoss()

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.291872  [    0/60000]
loss: 0.824696  [ 6400/60000]
loss: 0.427969  [12800/60000]
loss: 0.658392  [19200/60000]
loss: 0.550151  [25600/60000]
loss: 0.533732  [32000/60000]
loss: 0.483453  [38400/60000]
loss: 0.615152  [44800/60000]
loss: 0.598047  [51200/60000]
loss: 0.501011  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.495655 

Epoch 2
-------------------------------
Training mode: True
loss: 0.355374  [    0/60000]
loss: 0.443702  [ 6400/60000]
loss: 0.304433  [12800/60000]
loss: 0.530687  [19200/60000]
loss: 0.451318  [25600/60000]
loss: 0.447352  [32000/60000]
loss: 0.405146  [38400/60000]
loss: 0.559106  [44800/60000]
loss: 0.548364  [51200/60000]
loss: 0.480318  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.451472 

Epoch 3
-------------------------------
Training mode: True
loss: 0.298901  [    0/60000]
loss: 0.406026  [ 6400/60000]
loss: 0.298511